# 1. Super-cell writer

A code that generate 2D Serpent input files to perform FM precomputation. 

## 1.1. Content

* objectZoo: contains the  super-cells building blocks.

    * Pin(name, dimensions, materials)
    * Assm(name, pin_map, pitch, type_lattice)
    * Cell(name, assm_map, pitch, type_lattice)
    * Geometry(name, pin_set, assm_set, bc)
    * Material(name, density, temperature, composition, moder)
    * FissionMatrix(type_fm, limits)



* inputWriter: a collection of writers to create the Serpent input file. Inputs:
    * filePath, string
    * Geometry, object
    * Materials, objects List
    * Settings, dictionary
    * FissionMatrix, object.

## 1.2. Example 

In [1]:
# Import classes
from objectZoo import Pin, Assm, Cell, Geometry, Material, FissionMatrix
from inputWriter import SerpentWriter as sW

In [2]:
# FilePaths
filePath = 'miniCore.i'
libX = '/nv/hp22/dkotlyar6/data/Codes/DATA/sss_endfb7u.xsdata'
# Pin
radiiP = [0.410, 0.475, 1.26]
nPins = 17
# Assembly
pitchA = nPins*radiiP[-1]
pin_map = [['ff'] * nPins] * nPins
# SuperCell
assm_map = [['a1','a2']]
bc = ['vacuum', 'reflective'] # x and y direction

In [3]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# %                 MATERIALS                          %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
materials = []
# FUEL
composition = [['92235.09c', 0.02644492], ['92238.09c', 0.85505247], ['8016.09c', 0.11850261]]
fuel = Material('fuel', -10.3067, 900, composition, 'mass')
materials.append(fuel)
# WATER
composition = [['1001.06c', 0.6666667], ['8016.06c', 0.3333333]]
water = Material('water', -0.700452, 600, composition, 'molar', 'lwj3.11t ')
materials.append(water)
# Clad
composition = [['40000.06c', 1.0]]
clad = Material('clad', -6.5, 600, composition)
materials.append(clad)

In [4]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# %                 GEOMETRY                           %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

pin1 = Pin('ff', radiiP, [fuel.name, water.name, clad.name])
a1 = Assm('a1', pin_map, radiiP[-1])
a2 = Assm('a2', pin_map, radiiP[-1])
superCell = Cell('Super', assm_map, pitchA, bc)
geometry = Geometry('miniCore', [pin1], [a1, a2], superCell)

In [5]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# %                 SETTINGS                           %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
settings = {'pop': 100000,
            'active cycles': 100,
            'inactive cycles': 50,
            'k guess': 1.0,
            'ures': '92238.09c',
            'lib': libX
            }

In [6]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# %             FISSION MATRIX                         %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
fm = FissionMatrix('cartesian',
                   [-pitchA, pitchA, 2*nPins, pitchA/2, pitchA/2, nPins,
                    -1e37, 1e37, 1])

In [7]:
# EXECUTE
serpentInp = sW(filePath, geometry, materials, settings, fm)
serpentInp.write()

[['a1' 'a2']]
Appending fission matrix definition
Completed


## 1.3. Application to FM pregeneration stage

Current and future work